In [150]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import re
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('langgraph_workflow.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)



In [151]:
from typing import Literal
from enum import Enum
from langchain_core.language_models.chat_models import BaseChatModel

class Models(Enum):
    GEMMA3_12B_IT_QAT_Q4 = "hf.co/bartowski/google_gemma-3-12b-it-GGUF:Q4_K_M"
    GEMMA3_4B = "gemma3:4b"

    # return value of model when fetched
    def __str__(self):
        return self.value

def init_ollama_chat_model(model_name: Models):
    """
    Initialize the chat model from Ollama.
    """
    ollama_model_name = f"ollama:{model_name.value}"

    try:
        model : BaseChatModel = init_chat_model(ollama_model_name)
        logger.info(f"Model {model} initialized successfully.")
        return model
    except Exception as e:
        logger.error(f"Failed to initialize model {ollama_model_name}: {e}")
        raise

class ModelManager:
    """Class that lets me change model during runtime."""
    def __init__(self, initial_model: BaseChatModel):
        self.current_model = initial_model

    def switch_model(self, new_model: BaseChatModel):
        """Switch to a new chat model."""
        self.current_model = new_model
        logger.info(f"Switched model to {new_model}")

model_manager = ModelManager(
    init_ollama_chat_model(
        Models.GEMMA3_4B
    )
)

logger.info("Model initialized successfully")

In [152]:
def get_weather(location: str):
    """Returns weather info for a location.
    
    Args:
        location (str): The location to get the weather for.

    Returns:
        str: A string describing the weather.

    Example:
        >>> get_weather("San Francisco")
    """
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

def get_news():
    """Returns the latest news headlines.
    
    Returns:
        str: A string with the latest news headlines.

    Example:
        >>> get_news()
    """
    return "Latest news: AI is taking over the world!"

# Available tools dictionary
TOOLS = {
    "get_weather": get_weather,
    "get_news": get_news
}

In [ ]:
import inspect

def create_tool_description(tools: dict):
    """Creates a string description of available tools that we can pass to the model.
    
    Includes name, description and usage information for each tool.
    """
    
    # for each tool, return its docstring
    tool_descriptions = []
    for tool_name, tool_func in tools.items():
        signature = inspect.signature(tool_func)
        docstring = tool_func.__doc__
        tool_descriptions.append(f"def {tool_name}{signature}:\n\"\"\"{docstring}\n\"\"\"")
    return "\n".join(tool_descriptions)

# test that it works
logger.info(f"Tool description created:\n{create_tool_description(TOOLS)}")

In [ ]:
# Ensure you do not include any "." in the prompt - you will get errors during the function call!

def create_instruction_prompt(tool_description: str) -> str:
    instruction_prompt = f'''
    # Instructions
    You are a helpful conversational AI assistant.
    At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```.
    The python methods described below are imported and available, you can only use defined methods.
    ONLY use the ```tool_code``` format when absolutely necessary to answer the user's question.
    The generated code should be readable and efficient. 

    For questions that don't require any specific tools, just respond normally without tool calls.

    # Instructions for using tools:
    - Never use print statements. All tool outputs are automatically handled. Only use the tool call format as shown.
    - The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
    - When using a ```tool_call``` think step by step why and how it should be used. 
    - All tools will directly output a string into the `tool_output` variable. 

    The following Python methods are available:

    ```python
    {tool_description}
    ```

    # Example usage of tools:
    You can use a tool like this:
    ```tool_code
    my_tool("argument1", "argument2")
    ```
    - Where 'my_tool' is the name of the tool you want to call, and 'argument1', 'argument2' are the arguments you want to pass to the tool.

    # Bad example of tool usage:
    ```tool_code
    result = my_tool("argument1", "argument2")
    print(result)
    ```
    - This code will cause an error because the tool output is not being used correctly.

    ```tool_code
    print(my_tool("argument1", "argument2"))
    ```
    - This code will cause an error because the tool output is not being used correctly.
    '''

    return instruction_prompt

print(create_instruction_prompt(create_tool_description(TOOLS)))


# Instructions
You are a helpful conversational AI assistant.
At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```.
The python methods described below are imported and available, you can only use defined methods.
ONLY use the ```tool_code``` format when absolutely necessary to answer the user's question.
The generated code should be readable and efficient. 

For questions that don't require any specific tools, just respond normally without tool calls.

# Instructions for using tools:
- Never use print statements. All tool outputs are automatically handled. Only use the tool call format as shown.
- The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
- When using a ```tool_call``` think step by step why and how it should be used. 
- All tools will directly output a string into the `tool_output` variable. 

The following Python methods are available:

```python

In [155]:
def extract_tool_calls(text):
    """Extract tool calls from model output using regex parsing."""
    logger.info(f"[TOOL_PARSER] Starting tool extraction from text: {text[:500]}...")
    
    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        code = match.group(1).strip()
        logger.info(f"[TOOL_PARSER] Found tool code: {code}")
        
        try:
            logger.info(f"[TOOL_PARSER] Attempting to execute: {code}")
            logger.info(f"[TOOL_PARSER] Available tools: {list(TOOLS.keys())}")
            
            # Execute the tool call safely
            result = eval(code, {"__builtins__": {}}, TOOLS)
            logger.info(f"[TOOL_PARSER] Tool execution successful: {result}")
            
            return f'```tool_output\n{result}\n```'
        except Exception as e:
            logger.error(f"[TOOL_PARSER] Tool execution failed: {str(e)}")
            logger.error(f"[TOOL_PARSER] Error type: {type(e).__name__}")
            logger.error(f"[TOOL_PARSER] Code that failed: {code}")
            return f'```tool_output\nError: {str(e)}\n```'
    else:
        logger.info("[TOOL_PARSER] No tool_code blocks found in text")
        return None

In [ ]:
def react_agent(state: MessagesState):
    """Single ReAct agent that can generate responses and execute tools in a loop."""
    messages = state["messages"]
    logger.info(f"[REACT] Processing {len(messages)} messages")
    
    # Always include the system prompt for tool instructions
    system_prompt = create_instruction_prompt(create_tool_description(TOOLS))
    
    # Build conversation with system prompt
    conversation = [{"role": "system", "content": system_prompt}] + messages
    
    # Generate response
    response = model_manager.current_model.invoke(conversation)
    logger.info(f"[REACT] Model response: {response.content[:200]}...")
    
    # Check if response contains tool calls
    if '```tool_code' in str(response.content):
        logger.info("[REACT] Tool code detected - executing tools")
        
        # Execute the tool call
        tool_output = extract_tool_calls(response.content)
        
        if tool_output:
            logger.info(f"[REACT] Tool execution result: {tool_output}")
            
            # Extract the result from tool_output
            result_match = re.search(r'```tool_output\n(.*?)\n```', tool_output, re.DOTALL)
            if result_match:
                clean_result = result_match.group(1).strip()
                logger.info(f"[REACT] Clean result: {clean_result}")
                
                # Create a new response incorporating the tool result
                final_response_prompt = f"""Based on the tool result: {clean_result}
                
Please provide a helpful, natural response to the user incorporating this information. 
Do not include any tool code or technical details, just a conversational answer."""
                
                # Generate final response with tool result
                final_conversation = [
                    {"role": "system", "content": final_response_prompt},
                    {"role": "user", "content": messages[-1].content}
                ]
                
                final_response = model_manager.current_model.invoke(final_conversation)
                logger.info(f"[REACT] Final response with tool result: {final_response.content}")
                
                return {"messages": [final_response]}
    
    # No tool calls needed, return the response as-is
    logger.info("[REACT] No tool calls detected - returning response")
    return {"messages": [response]}

def should_continue_react(state: MessagesState):
    """Always end after the react agent processes the input."""
    return "end"

In [157]:
# Simplified ReAct graph setup 
builder = StateGraph(MessagesState)
builder.add_node("react", react_agent)

builder.add_edge(START, "react")
builder.add_edge("react", END)
graph = builder.compile()

In [158]:
# Enhanced utils for better debugging
def print_conversation(result):
    print("=== CONVERSATION FLOW ===")
    messages = result["messages"]
    
    for i, message in enumerate(messages):
        print(f"\n--- Message {i+1} ---")
        print(f"Type: {type(message).__name__}")
        print(f"Content: {message.content}")
        
        # Check if this message contains a tool call
        if '```tool_code' in str(message.content):
            print("🔧 TOOL CALL DETECTED")
            
            # Extract the tool code for debugging
            pattern = r"```tool_code\s*(.*?)\s*```"
            match = re.search(pattern, message.content, re.DOTALL)
            if match:
                code = match.group(1).strip()
                print(f"📝 Tool Code: {code}")
                
                # Try to execute and show result
                tool_output = extract_tool_calls(message.content)
                if tool_output:
                    print(f"🔧 Tool Result: {tool_output}")
                else:
                    print("❌ No tool output generated")
            else:
                print("❌ Could not extract tool code")
        
        # Show if this is a tool output
        if '```tool_output' in str(message.content):
            print("📊 TOOL OUTPUT DETECTED")
            
            # Extract the tool output for debugging
            pattern = r"```tool_output\n(.*?)\n```"
            match = re.search(pattern, message.content, re.DOTALL)
            if match:
                output = match.group(1).strip()
                print(f"📋 Output: {output}")
        
        print("-" * 50)

In [159]:
#excecute graph - test that chat history works
input_prompt = "You are a helpful assistant named Tim"
query = "What is your name? I am barry!"
messages = [
    SystemMessage(content=input_prompt),
    HumanMessage(content=query)
]
state : MessagesState = {"messages": messages}
result = graph.invoke({"messages": state["messages"]})
print_conversation(result)

=== CONVERSATION FLOW ===

--- Message 1 ---
Type: SystemMessage
Content: You are a helpful assistant named Tim
--------------------------------------------------

--- Message 2 ---
Type: HumanMessage
Content: What is your name? I am barry!
--------------------------------------------------

--- Message 3 ---
Type: AIMessage
Content: Hello Barry, my name is Tim! It’s nice to meet you.
--------------------------------------------------


In [160]:
# Test the simplified ReAct graph with weather query
input_prompt = "You are a helpful assistant named Tim"
query = "What is the weather in San Francisco?"
messages = [
    SystemMessage(content=input_prompt),
    HumanMessage(content=query)
]
state : MessagesState = {"messages": messages}
result = graph.invoke({"messages": state["messages"]})
print_conversation(result)

=== CONVERSATION FLOW ===

--- Message 1 ---
Type: SystemMessage
Content: You are a helpful assistant named Tim
--------------------------------------------------

--- Message 2 ---
Type: HumanMessage
Content: What is the weather in San Francisco?
--------------------------------------------------

--- Message 3 ---
Type: AIMessage
Content: It's 60 degrees and quite foggy here in San Francisco. A bit chilly and a lot of mist!
--------------------------------------------------


In [162]:
# Test the simplified ReAct graph with weather query
input_prompt = "You are a helpful assistant named Tim"
query = "What's the latest news?"
messages = [
    SystemMessage(content=input_prompt),
    HumanMessage(content=query)
]
state : MessagesState = {"messages": messages}
result = graph.invoke({"messages": state["messages"]})
print_conversation(result)

=== CONVERSATION FLOW ===

--- Message 1 ---
Type: SystemMessage
Content: You are a helpful assistant named Tim
--------------------------------------------------

--- Message 2 ---
Type: HumanMessage
Content: What's the latest news?
--------------------------------------------------

--- Message 3 ---
Type: AIMessage
Content: Wow, it looks like the latest news is pretty dramatic – apparently AI is taking over the world! It's a big topic, and I'm sure there's a lot more to it than that headline suggests, but definitely something to keep an eye on. Do you want me to try and find some more details about that?
--------------------------------------------------


In [164]:
# Disable logging for this cell   
#import logging                    
#logging.disable(logging.CRITICAL) 

test_prompts = {
    "What is the weather in San Francisco?": ["60 degrees", "foggy"],
    "What is the weather in New York City?": ["90 degrees", "sunny"],
}

def test(prompt: str = "What is the weather in San Francisco?"):
    # Test the simplified ReAct graph with weather query
    input_prompt = "You are a helpful assistant named Tim"
    query = prompt
    messages = [
        SystemMessage(content=input_prompt),
        HumanMessage(content=query)
    ]
    state : MessagesState = {"messages": messages}
    result = graph.invoke({"messages": state["messages"]})
    return result

def analyze_results(result, expected_outputs: list = ["60 degrees", "foggy"]):
    messages = result["messages"]

    # verify llm output a correct answer
    llm_output_is_correct = False

    ai_response = messages[-1]
    content = ai_response.content


    if expected_outputs[0] in content and expected_outputs[1] in content:
        llm_output_is_correct = True
        print(f"Tool output found in message: {content}")
    else:
        print(f"Tool output NOT found in message: {content}")

    if llm_output_is_correct:
        return True
    else:
        return False
    
def run_tests(with_model: Models):
    success_count = 0
    total_tests = 10

    model_manager.switch_model(init_ollama_chat_model(with_model))

    print("="* 30)
    print(f"Running tests with model {model_manager.current_model}")

    for i in range(total_tests):
        prompt = list(test_prompts.keys())[i % len(test_prompts)]
        result = test(prompt)
        success = analyze_results(result, expected_outputs=test_prompts[prompt])
        if success:
            success_count += 1

    print("="* 30)
    print(f"Tests completed: {success_count}/{total_tests} successful")
    print("Success rate: {:.2f}%".format((success_count / total_tests) * 100))
    print("="* 30)

run_tests(Models.GEMMA3_4B)
run_tests(Models.GEMMA3_12B_IT_QAT_Q4)

Running tests with model model='gemma3:4b'
Tool output found in message: It’s a pretty cozy day in San Francisco – it’s 60 degrees and quite foggy! You’ll definitely want a jacket on.
Tool output found in message: It’s 90 degrees and sunny in New York City! Sounds like a beautiful day to be outside.
Tool output found in message: It's looking pretty foggy and cool in San Francisco – it’s 60 degrees right now. You’ll definitely want a jacket!
Tool output found in message: It’s 90 degrees and sunny in New York City today! Sounds like a beautiful day to be outside.
Tool output found in message: It’s looking pretty foggy and cool over in San Francisco – it’s 60 degrees right now. Layers are definitely your friend today!
Tool output found in message: It’s 90 degrees and sunny in New York City! Sounds like a beautiful day to be outside.
Tool output found in message: It’s looking pretty foggy and cool here in San Francisco – it’s 60 degrees right now. You might want a jacket!
Tool output found

In [165]:
# define more tools in attempt to overwhelm model
def get_stock_price(symbol: str):
    """Returns the current stock price for a given symbol.
    
    Args:
        symbol (str): The stock symbol to get the price for.

    Returns:
        str: A string with the current stock price.

    Example:
        >>> get_stock_price("AAPL")
    """
    return f"The current price of {symbol} is $150.00."

def start_microwave(minutes: int):
    """Starts the microwave for a given number of minutes.
    
    Args:
        minutes (int): The number of minutes to run the microwave.

    Returns:
        str: A string confirming the microwave has started.

    Example:
        >>> start_microwave(2)
    """
    return f"The microwave has been started for {minutes} minutes."

def start_blender(speed: Literal['low', 'medium', 'high']):
    """Starts the blender at a given speed.
    
    Args:
        speed (str): The speed to run the blender at ('low', 'medium', 'high').

    Returns:
        str: A string confirming the blender has started.

    Example:
        >>> start_blender('medium')
    """
    return f"The blender has been started at {speed} speed."

def open_front_door():
    """Opens the front door.

    Returns:
        str: A string confirming the front door has been opened or not.

    Example:
        >>> open_front_door()
    """
    return "The front door has been opened."

TOOLS = {
    "get_weather": get_weather,
    "get_news": get_news,
    "get_stock_price": get_stock_price,
    "start_microwave": start_microwave,
    "start_blender": start_blender,
    "open_front_door": open_front_door
}

In [167]:
# Disable logging for this cell   
#import logging                    
#logging.disable(logging.CRITICAL) 

test_prompts = {
    "Can you open the front door?": ["The front door", "opened"],
    "What is the weather in San Francisco?": ["60 degrees", "foggy"],
    "Can you turn on the microwave for 2 minutes?": ["The microwave", "started for 2 minutes"],
    "Can you start the blender on medium speed?": ["The blender", "started at medium speed"],
    "What is the stock price of AAPL?": ["The current price of AAPL", "$150.00"],
    "Can you turn on the blender on high speed?": ["The blender", "started at high speed"]  
}

def test(prompt: str = "What is the weather in San Francisco?"):
    # Test the simplified ReAct graph with weather query
    input_prompt = "You are a helpful assistant named Tim"
    query = prompt
    messages = [
        SystemMessage(content=input_prompt),
        HumanMessage(content=query)
    ]
    state : MessagesState = {"messages": messages}
    result = graph.invoke({"messages": state["messages"]})
    return result

def analyze_results(result, expected_outputs: list = ["60 degrees", "foggy"]):
    messages = result["messages"]

    # verify llm output a correct answer
    llm_output_is_correct = False

    ai_response = messages[-1]
    content = ai_response.content


    if expected_outputs[0] in content and expected_outputs[1] in content:
        llm_output_is_correct = True
        print(f"Tool output found in message: {content}")
    else:
        print(f"Tool output NOT found in message: {content}")

    if llm_output_is_correct:
        return True
    else:
        return False
    
def run_tests(with_model: Models):
    success_count = 0
    total_tests = 10

    model_manager.switch_model(init_ollama_chat_model(with_model))

    print("="* 30)
    print(f"Running tests with model {model_manager.current_model}")

    for i in range(total_tests):
        prompt = list(test_prompts.keys())[i % len(test_prompts)]
        result = test(prompt)
        success = analyze_results(result, expected_outputs=test_prompts[prompt])
        if success:
            success_count += 1

    print("="* 30)
    print(f"Tests completed: {success_count}/{total_tests} successful")
    print("Success rate: {:.2f}%".format((success_count / total_tests) * 100))
    print("="* 30)

run_tests(Models.GEMMA3_4B)

Running tests with model model='gemma3:4b'
Tool output NOT found in message: ```tool_call
get_weather("front door")
```
Tool output found in message: It’s looking pretty foggy and cool here in San Francisco – it’s 60 degrees right now. You’ll definitely want a jacket!
Tool output NOT found in message: Sounds lovely out today – it’s 90 degrees and sunny! Absolutely, I’ll turn on the microwave for 2 minutes.
Tool output NOT found in message: Okay, I’m sorry, something went wrong – I need a location to get the weather! Could you please tell me where you’d like me to check the weather for?
Tool output NOT found in message: Okay, I understand you're trying to get the stock price of Apple (AAPL). Unfortunately, I'm having a little trouble right now – it seems like there's a problem with how I'm accessing that information. The error message indicates that I'm missing a basic command needed to get the data. 

Don't worry, I'm working on it! I’ll try again shortly to get you that information.
T